In [1]:
import pyterrier as pt
if not pt.started():
  pt.init()

C:\Users\Fortu\AppData\Local\Temp\ipykernel_26016\1742394333.py:2: DeprecationWarning: Call to deprecated function (or staticmethod) started. (use pt.java.started() instead) -- Deprecated since version 0.11.0.
  if not pt.started():
Java started and loaded: pyterrier.java, pyterrier.terrier.java [version=5.10 (build: craigm 2024-08-22 17:33), helper_version=0.0.8]
C:\Users\Fortu\AppData\Local\Temp\ipykernel_26016\1742394333.py:3: DeprecationWarning: Call to deprecated method pt.init(). Deprecated since version 0.11.0.
java is now started automatically with default settings. To force initialisation early, run:
pt.java.init() # optional, forces java initialisation
  pt.init()


In [3]:
files = pt.io.find_files('./nyt/nyt')
indexer = pt.TRECCollectionIndexer('c:/Users/Fortu/Downloads/Aut2024/Info 376/nyt_index',
                                    verbose=True, blocks=False, meta_tags={'docno':'docno'},
                                    overwrite=True)
indexref = indexer.index(files)
index = pt.IndexFactory.of(indexref)
print(index.getCollectionStatistics().toString())

3files [00:01,  1.47files/s]                    

11:46:41.582 [main] ERROR org.terrier.structures.indexing.Indexer -- Could not rename index
java.io.IOException: Rename of index structure file 'c:/Users/Fortu/Downloads/Aut2024/Info 376/nyt_index/data_1.direct.bf' (exists) to 'c:/Users/Fortu/Downloads/Aut2024/Info 376/nyt_index/data.direct.bf' (exists) failed - likely that source file is still open. Possible indexing bug?
	at org.terrier.structures.IndexUtil.renameIndex(IndexUtil.java:379)
	at org.terrier.structures.indexing.Indexer.index(Indexer.java:388)


3files [00:04,  1.36s/files]


Number of documents: 722
Number of terms: 19891
Number of postings: 215372
Number of fields: 0
Number of tokens: 357606
Field names: []
Positions:   false



In [11]:
import pandas as pd
queries = pd.DataFrame([["q1", "huskies football"], ["q2", "bush kerry"]], columns=["qid","query"])
queries

,qid,query
0,q1,huskies football
1,q2,bush kerry


In [ ]:
index = pt.IndexFactory.of("c:/Users/Fortu/Downloads/Aut2024/Info 376/nyt_index/data.properties")
tf_idf = pt.BatchRetrieve(index, wmodel="TF_IDF")
bm25 = pt.BatchRetrieve(index, wmodel="BM25")
Hiemstra = pt.BatchRetrieve(index, wmodel="Hiemstra_LM")

# retrieve documents for each query
tf_idf.transform(queries)

C:\Users\Fortu\AppData\Local\Temp\ipykernel_26016\3634909428.py:2: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  tf_idf = pt.BatchRetrieve(index, wmodel="TF_IDF")
C:\Users\Fortu\AppData\Local\Temp\ipykernel_26016\3634909428.py:3: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  bm25 = pt.BatchRetrieve(index, wmodel="BM25")
C:\Users\Fortu\AppData\Local\Temp\ipykernel_26016\3634909428.py:4: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  Hiemstra = pt.BatchRetrieve(index, wmodel="Hiemstra_LM")


,qid,docid,docno,rank,score,query
0,q1,88,NYT20000130.0100,0,8.055148,huskies football
1,q1,217,NYT20000131.0026,1,4.961224,huskies football
2,q1,21,NYT20000130.0022,2,3.492098,huskies football
3,q1,18,NYT20000130.0019,3,3.483695,huskies football
4,q1,19,NYT20000130.0020,4,3.483695,huskies football
...,...,...,...,...,...,...
191,q2,324,NYT20000131.0139,116,1.198013,bush kerry
192,q2,317,NYT20000131.0132,117,1.185011,bush kerry
193,q2,485,NYT20000131.0308,118,1.108579,bush kerry
194,q2,33,NYT20000130.0035,119,0.911068,bush kerry


In [15]:
qrels = pd.DataFrame([["q1","NYT20000130.0100",1],
                     ["q1","NYT20000130.0022",1],
                     ["q2","NYT20000131.0413",1],
                     ["q2","NYT20000131.0418",1]], columns=["qid","docno","label"])

# labelling the relevant documents by naming them in qrels

qrels

,qid,docno,label
0,q1,NYT20000130.0100,1
1,q1,NYT20000130.0022,1
2,q2,NYT20000131.0413,1
3,q2,NYT20000131.0418,1


In [17]:
# Calculate the mean average precision, R-precision, and reciprocal rank for each query
# using the tf_idf, bm25, and Hiemstra retrieval models

# MAP = mean average precision = average of the precision at each relevant document
# R-precision = precision at the Rth relevant document (R = number of relevant documents)
# Reciprocal Rank = 1/rank of the first relevant document

# map -> shows the average of the precision at each relevant document
# rprec -> shows the precision after R (number of relevant documents) has been retrieved
# recip_rank -> shows that relevant document is at the top of the list (shown by 1)

pt.Experiment (
    [tf_idf,bm25,Hiemstra],
    queries,
    qrels,
    eval_metrics=["map","Rprec","recip_rank"]
)

,name,map,Rprec,recip_rank
0,TerrierRetr(TF_IDF),0.916667,0.75,1.0
1,TerrierRetr(BM25),0.916667,0.75,1.0
2,TerrierRetr(Hiemstra_LM),0.850000,0.75,1.0


In [23]:
dataset = pt.get_dataset("vaswani")
indexer = pt.TRECCollectionIndexer("c:/Users/Fortu/Downloads/Aut2024/Info 376/vaswani_index")
indexref = indexer.index(dataset.get_corpus())
index = pt.IndexFactory.of(indexref)
print(index.getCollectionStatistics().toString())

Number of documents: 11429
Number of terms: 7756
Number of postings: 224573
Number of fields: 0
Number of tokens: 271581
Field names: []
Positions:   false



In [24]:
queries = pd.DataFrame([["4", "system of data coding for information transfer"],
                        ["6","number representation in binary machines"]],columns=["qid","query"])
queries

,qid,query
0,4,system of data coding for information transfer
1,6,number representation in binary machines


In [25]:
print(dataset.get_qrels())

qrels: 24.3kiB [00:00, 12.4MiB/s]                  

     qid  docno  label
0      1   1239      1
1      1   1502      1
2      1   4462      1
3      1   4569      1
4      1   5472      1
...   ..    ...    ...
2078  93   9875      1
2079  93   9956      1
2080  93  10497      1
2081  93  11191      1
2082  93  11318      1

[2083 rows x 3 columns]


In [26]:
tf_idf = pt.BatchRetrieve(index, wmodel="TF_IDF")
bm25 = pt.BatchRetrieve(index, wmodel="BM25")
Hiemstra = pt.BatchRetrieve(index, wmodel="Hiemstra_LM")

C:\Users\Fortu\AppData\Local\Temp\ipykernel_26016\1887074501.py:1: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  tf_idf = pt.BatchRetrieve(index, wmodel="TF_IDF")
C:\Users\Fortu\AppData\Local\Temp\ipykernel_26016\1887074501.py:2: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  bm25 = pt.BatchRetrieve(index, wmodel="BM25")
C:\Users\Fortu\AppData\Local\Temp\ipykernel_26016\1887074501.py:3: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  Hiemstra = pt.BatchRetrieve(index, wmodel="Hiemstra_LM")


In [30]:
# Hiemstra did better because it uses a language model approach.
# This approach estimates the probability of a document generating a query,
# which can be more effective for certain types of queries and collections.
# Specifically, it can handle term dependencies and variations better than
# traditional models like TF-IDF and BM25.

pt.Experiment (
  [tf_idf,bm25,Hiemstra],
  queries,
  dataset.get_qrels(),
  eval_metrics=["map","Rprec","recip_rank"]
)

,name,map,Rprec,recip_rank
0,TerrierRetr(TF_IDF),0.368055,0.4,1.0
1,TerrierRetr(BM25),0.363716,0.4,1.0
2,TerrierRetr(Hiemstra_LM),0.395372,0.4,1.0
